In [1]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import itertools

from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, ConfusionMatrix


In [2]:
# Defines the device on which the processing is going to take place.
device = torch.device("cpu")
# Preprocessing takes most of the time. 
# No significant reduction in training time if we use GPU as the model is not very big.


In [3]:
class BinarizeTransform:
    """
    A class to binarize the input MNIST data.
    """
    def __call__(self, img):
        # Values are between 0 and 1 so I have binarized with threshold of 0.5
        return (img>0.5).float()

# Transform to be applied on to the data immediately after loading from location.
transform = transforms.Compose([
    transforms.ToTensor(),
    BinarizeTransform()
])

# Load and transform MNIST data
mnist_data_train = datasets.MNIST(root="data/MNIST", train=True, download=True, transform=transform)
mnist_data_test = datasets.MNIST(root="data/MNIST", train=False, download=True, transform=transform)


In [4]:
train_size = int(0.9 * len(mnist_data_train))       # Size of the train split
val_size = len(mnist_data_train) - train_size       # Size of the validation split


train_data, val_data = random_split(mnist_data_train, [train_size, val_size])




In [5]:
len(train_data), len(val_data), len(mnist_data_test)

(54000, 6000, 10000)

In [5]:
# Dictionary of all the rows of each mask-set in every file 
json_data = {
    "365nm":{
        "I1":range(0, 5),
        "I2":range(10, 15),
        "I3":range(18, 23),
        "I4":range(25, 30)
    },
    "455nm":{
        "I1":range(0, 5),
        "I2":range(7, 12),
        "I3":range(14, 19),
        "I4":range(21, 26)
    },
    "White":{
        "I1":range(0, 5),
        "I2":range(9, 14),
        "I3":range(16, 21),
        "I4":range(24, 29)
    }
}

In [ ]:
filename = "365nm"  # Excel document name from which data has to be extracted.
path = "data/"+filename+".xlsx" # Excel doc path

df = pd.read_excel(path, usecols='B:Q') # Read the excel sheet
# Break the sheet down into different `DataFrame`s for every Mask-set in the table
tables = [df.iloc[json_data[filename][key]].copy().reset_index(drop=True) for key in list(json_data[filename].keys())]

# Mask sets to be used
mask_sets = [3]
# Randomly initialize the device states (Uniform distribution over all the states)
device_indices = torch.randint(0, 5, (len(mask_sets), 196))


In [ ]:

class CustomMNISTDataset(Dataset):
    """Dataset object to save the preprocessed mnist dataset

    Parameters -
    mnist_data : `torch.utils.data.dataset`
                Contains images and corresponding labels of MNIST dataset               
    tables : `list[pd.DataFrame]`
                conductance tables for every mask-set.
    device_indices : `torch.tensor`
                Initial conductance states of every device. Shape of `((len(mask_sets), 196))`
    mask_sets : `List[int]`
                List of all the mask-sets to be used

    Attributes - 

    mask_sets : `List[int]`
                A list of all the mask-sets used for simulation.
    processed_data : `torch.tensor`
                MNIST images after preprocessing. Shape of ((N_samples, 1, 196 * len(mask_sets)))
    labels : `torch.tensor`
                label of each corresponding image. Shape of ((N_samples, num_classes))   (num_classes = 10)
    device_indices : List[List]
                Initial states of each device for every mask-set
    """
    def __init__(self, mnist_data, tables, device_indices, mask_sets ):
        self.mask_sets = mask_sets
        self.processed_data = []
        self.labels = []

        self.device_indices = device_indices.int().tolist()
        
        # Preprocessing step. Same as discussed in the paper given.
        for idx in tqdm(range(len(mnist_data))):
            image, label = mnist_data[idx]
            image = image.reshape(1, 196, 4)
            image_combined = (
                image[:, :, 0] * 1000 + 
                image[:, :, 1] * 100 + 
                image[:, :, 2] * 10 + 
                image[:, :, 3]
            )
            # x = []
            # for m_idx, mask in enumerate(self.mask_sets):
            #     for i, device in enumerate(self.device_indices[m_idx]):                
            #         x.append(tables[mask].loc[int(device), int(image_combined[0][i])]*1e9)
            x = [
                tables[mask].loc[int(device), int(image_combined[0][i])] * 1e9
                for mask_idx, mask in enumerate(self.mask_sets)
                for i, device in enumerate(self.device_indices[mask_idx])
            ]*1e9
            
            self.processed_data.append(x)
            self.labels.append(label)
        
        # Convert the lists to `torch.tensor`
        self.processed_data = torch.tensor(self.processed_data)
        self.labels = torch.tensor(self.labels)

    def __len__(self):
        """ Function to access length of the dataset object.

        Returns: `int`
                    Length of the dataset object
        """
        return len(self.processed_data)

    def __getitem__(self, idx):
        """ Returns element's image and label at a given index.
        Parameters - 
        idx : int
            index of the element to be accessed

        Returns - torch.tensor, torch.tensor
        """
        return self.processed_data[idx], self.labels[idx]

In [6]:
class CustomDataset(Dataset):
    def __init__(self, data, combined_table:pd.DataFrame, device_masks, mask_set):
        self.processed_data = []
        self.labels = []

        for idx in tqdm(range(len(data))):
            image, label = data[idx]
            image = image.reshape(196, 4)
            optical_pulses = (
                image[ :, 0] * 1000 + 
                image[ :, 1] * 100 + 
                image[ :, 2] * 10 + 
                image[ :, 3]
            ).repeat(len(mask_set))
            row_indices = torch.tensor([(device_masks[i]+mask_set[i]*5).tolist() for i in range(len(mask_set))]).flatten()
            column_indices = combined_table.columns.get_indexer(optical_pulses.tolist())
            
            self.processed_data.append(   combined_table.values[row_indices, column_indices] *1e9 )
            self.labels.append(label)
        self.processed_data = torch.tensor(np.array(self.processed_data)).to(device=device)
        self.labels = torch.tensor(self.labels).to(device=device)

    def __len__(self):
        return self.processed_data.shape[0]
    def __getitem__(self, index):
        return self.processed_data[index], self.labels[index]


In [19]:
# Load and preprocess training dataset
train_dataset = CustomMNISTDataset(train_data, tables=tables, mask_sets=mask_sets, device_indices=device_indices)

100%|██████████| 54000/54000 [03:03<00:00, 294.20it/s]


In [20]:
validation_dataset = CustomMNISTDataset(val_data, tables=tables, mask_sets=mask_sets, device_indices=device_indices)

100%|██████████| 6000/6000 [00:19<00:00, 300.09it/s]


In [21]:
test_dataset = CustomMNISTDataset(mnist_data_test, tables=tables, mask_sets=mask_sets, device_indices=device_indices)

100%|██████████| 10000/10000 [00:33<00:00, 294.44it/s]


In [7]:

# Model hyperparameters
BATCH_SIZE = 64
EPOCHS = 100
learning_rate = 0.001

In [22]:

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
class ReadoutLayer(nn.Module):
    """Readout layer.
    Parameters - 

    Attributes - 
    fc : `nn.Linear`
            Fully connected layer to be trained for reservoir computing.
    activation : `nn.functional.leaky_relu`
            Activation layer to be applied
    
    softmax : `nn.Softmax`
            Softmax activation function to get the one-hot encoded results.
    
    """
    def __init__(self, input_size):
        # super function to initialize the constructors of the parent classes.
        super(ReadoutLayer, self).__init__()
        # Class Attributes
        self.fc = nn.Linear(input_size, 10) 
        self.activation = nn.functional.leaky_relu
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        """Forward method to be executed on function call.
        Parameters - 
        x : torch.tensor
            Shape( Batch_size, 196 * len(mask_sets) )
        
        Returns : torch.tensor
            Shape( Batch_size, 10 )
        """
        x = self.fc(x)
        x = self.activation(x)
        x = self.softmax(x)
        return x

In [9]:
def get_metrics(filename, mask_sets, device_indices):
    # filename = "365nm"  # Excel document name from which data has to be extracted.
    path = "data/"+filename+".xlsx" # Excel doc path

    df = pd.read_excel(path, usecols='B:Q') # Read the excel sheet
    # Break the sheet down into different `DataFrame`s for every Mask-set in the table
    tables = [df.iloc[json_data[filename][key]].copy().reset_index(drop=True) for key in list(json_data[filename].keys())]
    combined_table = pd.concat(tables, axis=0)

    # train_dataset = CustomMNISTDataset(train_data, tables=tables, mask_sets=mask_sets, device_indices=device_indices)
    # validation_dataset = CustomMNISTDataset(val_data, tables=tables, mask_sets=mask_sets, device_indices=device_indices)
    # test_dataset = CustomMNISTDataset(mnist_data_test, tables=tables, mask_sets=mask_sets, device_indices=device_indices)
    
    train_dataset = CustomDataset(train_data, combined_table, device_indices, mask_sets)
    validation_dataset = CustomDataset(val_data, combined_table, device_indices, mask_sets)
    test_dataset = CustomDataset(mnist_data_test, combined_table, device_indices, mask_sets)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Model initialization
    model = ReadoutLayer(len(mask_sets)*196)
    # Loss function, Categorical crossentropy loss for multi-class classification problem
    criterion = nn.CrossEntropyLoss()
    # Optimizer to update the gradients.
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    val_accuracy, val_precision, val_recall, val_fscore = [], [], [], []
    # Evaluation metrics
    accuracy = Accuracy(task="multiclass", num_classes=10).to(device)
    precision = Precision(task="multiclass", num_classes=10, average='macro').to(device)
    recall = Recall(task="multiclass", num_classes=10, average='macro').to(device)
    f1_score = F1Score(task="multiclass", num_classes=10, average='macro').to(device)
    # Class-wise Confusion matrix
    confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=10).to(device)

    # Training loop
    for epoch in range(EPOCHS):
        # Training phase
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move to device
            outputs = model(images.float())  # Forward pass
            loss = criterion(outputs, labels)  # Loss calculation
            optimizer.zero_grad()
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')
        
        # Validation phase
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images.float())
                preds = outputs.argmax(dim=1)

                # Update metrics
                accuracy.update(preds, labels)
                precision.update(preds, labels)
                recall.update(preds, labels)
                f1_score.update(preds, labels)

            # Print validation metrics
            print(f'Validation Accuracy: {accuracy.compute().item():.4f} Precision: {precision.compute().item():.4f} ', end="")
            print(f'Validation Recall: {recall.compute().item():.4f} F1 Score: {f1_score.compute().item():.4f}')

            # Updating the list to save current metrics
            val_accuracy.append(accuracy.compute().item())
            val_precision.append(precision.compute().item())
            val_recall.append(recall.compute().item())
            val_fscore.append(f1_score.compute().item())

            # Reset metrics for the next epoch
            accuracy.reset()
            precision.reset()
            recall.reset()
            f1_score.reset()
            confusion_matrix.reset()

    
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            # Move images and labels to GPU
            images, labels = images.to(device), labels.to(device)

            outputs = model(torch.tensor(images, dtype=torch.float32))
            _, predicted = torch.max(outputs, 1)

            # Append predictions and labels for metric calculations
            all_preds.append(predicted)
            all_labels.append(labels)


    # Concatenate all predictions and labels
    all_preds = torch.cat(all_preds).to(device)
    all_labels = torch.cat(all_labels).to(device)

    # Calculate metrics
    test_accuracy = accuracy(all_preds, all_labels)
    test_precision = precision(all_preds, all_labels)
    test_recall = recall(all_preds, all_labels)
    test_f1 = f1_score(all_preds, all_labels)
    test_confusion_matrix = confusion_matrix(all_preds, all_labels)

    print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
    print(f'Test Precision: {test_precision*100:.4f}%')
    print(f'Test Recall: {test_recall*100:.4f}%')
    print(f'Test F1 Score: {test_f1:.4f}')
    print("Confusion Matrix:")
    print(test_confusion_matrix)

    save_path = "data/mnist_results_debug/" + filename + '_' + ''.join(map(str, mask_sets)) + '.npz'
    np.savez(save_path,
            predictions = all_preds,
            labels = all_labels,
            validation_accuracy = val_accuracy,
            validation_precision = val_precision,
            validation_recall = val_recall,
            validation_fscore = val_fscore
            
    )
    return test_accuracy, test_precision, test_recall, test_f1



In [ ]:
accuracy, precision, recall, fscore = get_metrics(filename="365nm", mask_sets=[0, 1, 2, 3], device_indices=torch.randint(0, 5, (4, 196)))

100%|██████████| 10000/10000 [00:06<00:00, 1579.26it/s]


Epoch [1/100], Loss: 1.7824
Validation Accuracy: 0.6785 Precision: 0.4860 Validation Recall: 0.6548 F1 Score: 0.5548
Epoch [2/100], Loss: 1.6760
Validation Accuracy: 0.7547 Precision: 0.6199 Validation Recall: 0.7330 F1 Score: 0.6656
Epoch [3/100], Loss: 1.6372
Validation Accuracy: 0.8217 Precision: 0.7532 Validation Recall: 0.8107 F1 Score: 0.7764
Epoch [4/100], Loss: 1.6530
Validation Accuracy: 0.8277 Precision: 0.7530 Validation Recall: 0.8172 F1 Score: 0.7811
Epoch [5/100], Loss: 1.6649
Validation Accuracy: 0.8272 Precision: 0.7568 Validation Recall: 0.8163 F1 Score: 0.7812
Epoch [6/100], Loss: 1.6050
Validation Accuracy: 0.8325 Precision: 0.7621 Validation Recall: 0.8233 F1 Score: 0.7875
Epoch [7/100], Loss: 1.5860
Validation Accuracy: 0.8308 Precision: 0.7592 Validation Recall: 0.8209 F1 Score: 0.7852
Epoch [8/100], Loss: 1.6300
Validation Accuracy: 0.8342 Precision: 0.7657 Validation Recall: 0.8240 F1 Score: 0.7895
Epoch [9/100], Loss: 1.6615
Validation Accuracy: 0.8328 Precisio

C:\Users\adith\AppData\Local\Temp\ipykernel_26748\2658863303.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(images, dtype=torch.float32))


Test Accuracy: 83.95%
Test Precision: 77.3635%
Test Recall: 83.9154%
Test F1 Score: 0.8000
Confusion Matrix:
tensor([[ 962,    0,    0,    5,    0,    3,    4,    2,    4,    0],
        [   0, 1112,    4,    2,    0,    3,    4,    1,    9,    0],
        [  15,    4,  916,   15,   17,    3,   15,   13,   34,    0],
        [   3,    1,   15,  922,    3,   21,    1,   15,   29,    0],
        [   2,    2,    2,    2,  949,    1,   11,    9,    4,    0],
        [  10,    5,    3,   41,   15,  763,   13,    4,   38,    0],
        [  14,    4,    8,    1,    8,   15,  903,    2,    3,    0],
        [   2,   10,   21,    5,   15,    0,    1,  974,    0,    0],
        [   2,    6,    3,   20,   17,   14,    9,    9,  894,    0],
        [   8,    5,    0,   12,  568,   25,    1,  325,   65,    0]])


In [29]:

def get_all_combinations(lis):
    all_combinations = []
    for r in range(1, len(lis)+1):
        combs = list(itertools.combinations(lis, r))
        all_combinations.extend(combs)
    return [list(ele) for ele in all_combinations]

In [ ]:
metrics_json = {
    "White":{},
    "365nm":{},
    "455nm":{}
}
mask_sets = [0]
combinations = get_all_combinations(mask_sets)
for filename in ["White", "455nm"]:
    for ms in combinations:

        dev_ind = torch.randint(0, 5, (len(ms), 196))

        accuracy, precision, recall, fscore = get_metrics(filename=filename, mask_sets=ms, device_indices=dev_ind)
        metrics_json[filename][''.join(map(str, ms)) ] = {
            "Accuracy":accuracy.item(),
            "Precision":precision.item(),
            "Recall":recall.item(),
            "F-score":fscore.item()
            
        }

            

100%|██████████| 10000/10000 [00:38<00:00, 263.05it/s]


Epoch [1/100], Loss: 2.0585
Validation Accuracy: 0.5460 Precision: 0.3341 Validation Recall: 0.5283 F1 Score: 0.4063
Epoch [2/100], Loss: 2.0079
Validation Accuracy: 0.5568 Precision: 0.3421 Validation Recall: 0.5395 F1 Score: 0.4155
Epoch [3/100], Loss: 1.8727
Validation Accuracy: 0.5615 Precision: 0.3422 Validation Recall: 0.5438 F1 Score: 0.4179
Epoch [4/100], Loss: 1.8594
Validation Accuracy: 0.6467 Precision: 0.4642 Validation Recall: 0.6328 F1 Score: 0.5318
Epoch [5/100], Loss: 1.7686
Validation Accuracy: 0.6928 Precision: 0.5736 Validation Recall: 0.6793 F1 Score: 0.6083
Epoch [6/100], Loss: 1.7550
Validation Accuracy: 0.7215 Precision: 0.5876 Validation Recall: 0.7080 F1 Score: 0.6387
Epoch [7/100], Loss: 1.7963
Validation Accuracy: 0.7245 Precision: 0.5893 Validation Recall: 0.7111 F1 Score: 0.6409
Epoch [8/100], Loss: 1.8111
Validation Accuracy: 0.7262 Precision: 0.5918 Validation Recall: 0.7128 F1 Score: 0.6428
Epoch [9/100], Loss: 1.7984
Validation Accuracy: 0.7248 Precisio

C:\Users\adith\AppData\Local\Temp\ipykernel_27520\3621217840.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(images, dtype=torch.float32))


Test Accuracy: 80.95%
Test Precision: 74.4073%
Test Recall: 80.8918%
Test F1 Score: 0.7703
Confusion Matrix:
tensor([[ 943,    0,    4,    6,    2,   11,    7,    1,    6,    0],
        [   0, 1076,    6,    8,    2,    3,    4,    1,   35,    0],
        [  16,    9,  878,   26,   16,    3,   23,   14,   47,    0],
        [   9,    4,   29,  874,    1,   31,    6,   18,   38,    0],
        [   5,    2,    2,    0,  942,    4,   14,    6,    7,    0],
        [  12,    6,    8,   51,   21,  711,   19,   17,   47,    0],
        [  11,    7,    8,    3,   12,   17,  897,    0,    3,    0],
        [   2,   15,   26,   10,   17,    5,    1,  950,    2,    0],
        [   9,   11,   12,   26,   20,   42,   14,   16,  824,    0],
        [  14,    8,    4,    8,  577,   28,    3,  305,   62,    0]])


100%|██████████| 10000/10000 [00:32<00:00, 310.49it/s]


Epoch [1/100], Loss: 1.8341
Validation Accuracy: 0.6545 Precision: 0.5313 Validation Recall: 0.6477 F1 Score: 0.5809
Epoch [2/100], Loss: 1.7685
Validation Accuracy: 0.6753 Precision: 0.5465 Validation Recall: 0.6674 F1 Score: 0.5986
Epoch [3/100], Loss: 1.8306
Validation Accuracy: 0.6842 Precision: 0.5575 Validation Recall: 0.6764 F1 Score: 0.6081
Epoch [4/100], Loss: 1.7645
Validation Accuracy: 0.6882 Precision: 0.5593 Validation Recall: 0.6814 F1 Score: 0.6114
Epoch [5/100], Loss: 1.9042
Validation Accuracy: 0.6923 Precision: 0.5679 Validation Recall: 0.6857 F1 Score: 0.6170
Epoch [6/100], Loss: 1.6923
Validation Accuracy: 0.7732 Precision: 0.7035 Validation Recall: 0.7656 F1 Score: 0.7313
Epoch [7/100], Loss: 1.6448
Validation Accuracy: 0.7762 Precision: 0.7057 Validation Recall: 0.7687 F1 Score: 0.7338
Epoch [8/100], Loss: 1.7582
Validation Accuracy: 0.7807 Precision: 0.7128 Validation Recall: 0.7735 F1 Score: 0.7392
Epoch [9/100], Loss: 1.7323
Validation Accuracy: 0.7830 Precisio

C:\Users\adith\AppData\Local\Temp\ipykernel_27520\3621217840.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(images, dtype=torch.float32))


In [12]:
# Specify the file name
filename = 'data/metrics.xlsx'

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
    for wavelength, results in metrics_json.items():
        # Flatten the data for each wavelength
        flattened_data = []
        for index, metrics in results.items():
            if metrics:  # Check if the metrics are not empty
                flattened_data.append({
                    'Index': index,
                    'Accuracy': metrics['Accuracy'],  # Convert tensor to float
                    'Precision': metrics['Precision'],
                    'Recall': metrics['Recall'],
                    'F-score': metrics['F-score']
                })
        
        # Create a DataFrame for the current wavelength
        if flattened_data:  # Check if there's data to save
            df = pd.DataFrame(flattened_data)
            # Write the DataFrame to a specific sheet named after the wavelength
            df.to_excel(writer, sheet_name=wavelength, index=False)

print("Data saved to", filename)

Data saved to data/metrics.xlsx


In [24]:
# Model initialization
model = ReadoutLayer(len(mask_sets)*196)
# Loss function, Categorical crossentropy loss for multi-class classification problem
criterion = nn.CrossEntropyLoss()
# Optimizer to update the gradients.
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
val_accuracy, val_precision, val_recall, val_fscore = [], [], [], []
# Evaluation metrics
accuracy = Accuracy(task="multiclass", num_classes=10).to(device)
precision = Precision(task="multiclass", num_classes=10, average='macro').to(device)
recall = Recall(task="multiclass", num_classes=10, average='macro').to(device)
f1_score = F1Score(task="multiclass", num_classes=10, average='macro').to(device)

# Class-wise Confusion matrix
confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=10).to(device)

# Training loop
for epoch in range(EPOCHS):
    # Training phase
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move to device
        outputs = model(images.float())  # Forward pass
        loss = criterion(outputs, labels)  # Loss calculation
        optimizer.zero_grad()
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')
    
    # Validation phase
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images.float())
            preds = outputs.argmax(dim=1)

            # Update metrics
            accuracy.update(preds, labels)
            precision.update(preds, labels)
            recall.update(preds, labels)
            f1_score.update(preds, labels)

        # Print validation metrics
        print(f'Validation Accuracy: {accuracy.compute().item():.4f} Precision: {precision.compute().item():.4f} ', end="")
        print(f'Validation Recall: {recall.compute().item():.4f} F1 Score: {f1_score.compute().item():.4f}')

        # Updating the list to save current metrics
        val_accuracy.append(accuracy.compute().item())
        val_precision.append(precision.compute().item())
        val_recall.append(recall.compute().item())
        val_fscore.append(f1_score.compute().item())

        # Reset metrics for the next epoch
        accuracy.reset()
        precision.reset()
        recall.reset()
        f1_score.reset()
        confusion_matrix.reset()


Epoch [1/100], Loss: 1.7956
Validation Accuracy: 0.7177 Precision: 0.5817 Validation Recall: 0.7140 F1 Score: 0.6382
Epoch [2/100], Loss: 1.7219
Validation Accuracy: 0.7248 Precision: 0.5893 Validation Recall: 0.7213 F1 Score: 0.6454
Epoch [3/100], Loss: 1.6405
Validation Accuracy: 0.8622 Precision: 0.8630 Validation Recall: 0.8614 F1 Score: 0.8613
Epoch [4/100], Loss: 1.6338
Validation Accuracy: 0.8715 Precision: 0.8713 Validation Recall: 0.8708 F1 Score: 0.8704
Epoch [5/100], Loss: 1.5685
Validation Accuracy: 0.8763 Precision: 0.8761 Validation Recall: 0.8752 F1 Score: 0.8756
Epoch [6/100], Loss: 1.5794
Validation Accuracy: 0.8778 Precision: 0.8777 Validation Recall: 0.8768 F1 Score: 0.8770
Epoch [7/100], Loss: 1.5311
Validation Accuracy: 0.8733 Precision: 0.8728 Validation Recall: 0.8724 F1 Score: 0.8724
Epoch [8/100], Loss: 1.6506
Validation Accuracy: 0.8775 Precision: 0.8770 Validation Recall: 0.8766 F1 Score: 0.8766
Epoch [9/100], Loss: 1.5100
Validation Accuracy: 0.8765 Precisio

KeyboardInterrupt: 

In [15]:

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        # Move images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        outputs = model(torch.tensor(images, dtype=torch.float32))
        _, predicted = torch.max(outputs, 1)

        # Append predictions and labels for metric calculations
        all_preds.append(predicted)
        all_labels.append(labels)


# Concatenate all predictions and labels
all_preds = torch.cat(all_preds).to(device)
all_labels = torch.cat(all_labels).to(device)

# Calculate metrics
test_accuracy = accuracy(all_preds, all_labels)
test_precision = precision(all_preds, all_labels)
test_recall = recall(all_preds, all_labels)
test_f1 = f1_score(all_preds, all_labels)
test_confusion_matrix = confusion_matrix(all_preds, all_labels)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Precision: {test_precision*100:.4f}%')
print(f'Test Recall: {test_recall*100:.4f}%')
print(f'Test F1 Score: {test_f1:.4f}')
print("Confusion Matrix:")
print(test_confusion_matrix)

        

C:\Users\adith\AppData\Local\Temp\ipykernel_29648\247194469.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(images, dtype=torch.float32))


Test Accuracy: 86.93%
Test Precision: 86.6844%
Test Recall: 86.7090%
Test F1 Score: 0.8669
Confusion Matrix:
tensor([[ 938,    1,    4,    4,    1,   13,   14,    2,    3,    0],
        [   0, 1099,    8,    3,    1,    4,    7,    3,   10,    0],
        [  13,    3,  884,   30,   14,   15,   18,   16,   37,    2],
        [   4,    6,   43,  837,    2,   48,    7,   16,   40,    7],
        [   5,    2,   13,    0,  828,    8,   11,    8,    8,   99],
        [  10,    3,   17,   61,   20,  693,   25,   12,   35,   16],
        [  15,    3,   14,    1,   13,   21,  876,    7,    7,    1],
        [   4,    8,   22,    6,   13,    3,    3,  924,    7,   38],
        [   6,   17,   16,   50,   13,   62,    7,   19,  765,   19],
        [   7,    5,    1,    8,   54,   16,    1,   44,   24,  849]])


In [16]:
save_path = "data/mnist_results/" + filename + '_' + ''.join(map(str, mask_sets)) + '.npz'
np.savez(save_path,
         predictions = all_preds,
         labels = all_labels,
         validation_accuracy = val_accuracy,
         validation_precision = val_precision,
         validation_recall = val_recall,
         validation_fscore = val_fscore
         
)

In [17]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader
# from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, ConfusionMatrix

# device = torch.device("cpu")

# # Hyperparameters
# batch_size = 64
# learning_rate = 0.001
# epochs = 5

# # MNIST dataset
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# train_dataset = datasets.MNIST(root='./data/MNIST', train=True, transform=transform, download=False)
# test_dataset = datasets.MNIST(root='./data/MNIST', train=False, transform=transform)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# # Define the single-layer MLP model
# class SingleLayerMLP(nn.Module):
#     def __init__(self):
#         super(SingleLayerMLP, self).__init__()
#         self.fc = nn.Linear(784, 10)  # Input layer (784) to output layer (10 classes)

#     def forward(self, x):
#         x = x.view(-1, 28*28)  # Flatten the input
#         x = self.fc(x)
#         return x

# # Initialize model, loss function, and optimizer
# model = SingleLayerMLP()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Initialize metrics
# accuracy = Accuracy(task="multiclass", num_classes=10).to(device)
# precision = Precision(task="multiclass", num_classes=10, average='macro').to(device)
# recall = Recall(task="multiclass", num_classes=10, average='macro').to(device)
# f1_score = F1Score(task="multiclass", num_classes=10, average='macro').to(device)
# confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=10).to(device)

# # Training loop
# for epoch in range(epochs):
#     model.train()
#     for images, labels in train_loader:
#         # Move images and labels to GPU
#         images, labels = images.to(device), labels.to(device)

#         # Forward pass
#         outputs = model(images)
#         loss = criterion(outputs, labels)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# # Testing loop
# model.eval()
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for images, labels in test_loader:
#         # Move images and labels to GPU
#         images, labels = images.to(device), labels.to(device)

#         outputs = model(images)
#         _, predicted = torch.max(outputs, 1)

#         # Append predictions and labels for metric calculations
#         all_preds.append(predicted)
#         all_labels.append(labels)

# # Concatenate all predictions and labels
# all_preds = torch.cat(all_preds)
# all_labels = torch.cat(all_labels)

# # Calculate metrics
# test_accuracy = accuracy(all_preds, all_labels)
# test_precision = precision(all_preds, all_labels)
# test_recall = recall(all_preds, all_labels)
# test_f1 = f1_score(all_preds, all_labels)
# test_confusion_matrix = confusion_matrix(all_preds, all_labels)

# print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
# print(f'Test Precision: {test_precision*100:.4f}%')
# print(f'Test Recall: {test_recall*100:.4f}%')
# print(f'Test F1 Score: {test_f1:.4f}')
# print("Confusion Matrix:")
# print(test_confusion_matrix)